In [ ]:
import pandas_utils as pu
import pandas as pd

In [ ]:
filename = r"C:\git\msn_library\data\gnpslib\library_cleanup\gnps_library.parquet"

## Combining GNPS chunks, adding classyfire results from previous run, adding nina_source

In [ ]:
df = pu.combine_chunks(filename)
# pu.save_dataframe(df, filename)
# pu.save_dataframe(df, pu.add_filename_suffix(filename, "cleaned", "csv"))

In [ ]:
df = pu.read_dataframe(r"C:\git\msn_library\data\gnpslib\library_cleanup\gnps_library.parquet")
df[["compound_name", "input_name", "inchikey"]].sort_values(by=["inchikey"])
df["gnps_libid"] = df["gnps_libid"].str.split(", ")
df = df.explode("gnps_libid").reset_index(drop=True)
df

In [ ]:
df[["compound_name","input_name", "inchikey"]].sort_values(by=["inchikey"])

In [ ]:
df[df["gnps_libid"].duplicated()]

In [ ]:
df_nina = pu.read_dataframe(r"C:\git\msn_library\data\nina\20230616_merged_metadata_drug_Nina_Corinna_DOA_GNPS.parquet")
df_nina = df_nina.drop(df_nina.columns[0], axis=1)
df_nina

In [ ]:
df_class = df_nina[[col for col in df_nina.columns if "classyfire" in col or col == "gnps_libid"]].copy()
df_class

In [ ]:
df_class_add = pd.merge(df, df_class, on="gnps_libid", how="outer")
df_class_add
# pu.save_dataframe(df_class_add, filename)
# pu.save_dataframe(df_class_add, pu.add_filename_suffix(filename, "cleaned", "csv"))

In [ ]:
df_nina_source = df_nina[["nina_source", 'gnps_libid']].copy()
df_nina_source

In [ ]:
df_nina_source_added = pd.merge(df_class_add, df_nina_source, on="gnps_libid", how="outer")
df_nina_source_added["missing_values"] = df_nina_source_added.isnull().sum(axis=1)
df_nina_source_addeddf_nina_source_added = pd.merge(df_class_add, df_nina_source, on="gnps_libid", how="outer")
df_nina_source_added["missing_values"] = df_nina_source_added.isnull().sum(axis=1)
df_nina_source_added

In [ ]:
df_nina_source_added.groupby("nina_source").count()

In [ ]:
pu.save_dataframe(df_nina_source_added, r"C:\git\msn_library\data\nina\20230806_merged_metadata_drug_Nina_Corinna_DOA_GNPS.csv")
pu.save_dataframe(df_nina_source_added, r"C:\git\msn_library\data\nina\20230806_merged_metadata_drug_Nina_Corinna_DOA_GNPS.parquet")

## Combine same entries, keep all gnps_libid

In [ ]:
df_nina_source_added = df_nina_source_added.sort_values("missing_values", ascending=True)
grouped = df_nina_source_added.groupby(['inchikey', 'inchi', 'isomeric_smiles', 'compound_name'])
df_grouped = grouped.first()
df_grouped["gnps_libid"] = grouped['gnps_libid'].apply(', '.join)
df_grouped = df_grouped.reset_index()
df_grouped["entries"] = [str(lib).count(",") + 1 for lib in df_grouped["gnps_libid"]]
df_grouped

In [ ]:
pu.save_dataframe(df_grouped, r"C:\git\msn_library\data\nina\20230806_gnps_grouped_dataframe.csv")
pu.save_dataframe(df_grouped, r"C:\git\msn_library\data\nina\20230806_gnps_grouped_dataframe.parquet")

In [ ]:
df_grouped.drop_duplicates("inchikey")

In [ ]:
df_grouped.groupby("inchikey").count()

In [ ]:
df_grouped.loc[df_grouped["inchikey"] == "MVMSCBBUIHUTGJ-UHFFFAOYSA-N"][["inchikey", "inchi", "isomeric_smiles", "compound_name", "input_name", "drugcentral_id", "pubchem_cid", "drugcentral_compound_name"]]

In [ ]:
missing = df_nina_source_added.loc[df_nina_source_added["missing_values"] == 142][
    ["gnps_libid", "nina_source", "smiles", "any_phase", "missing_values"]]

In [ ]:
pu.read_dataframe(r"C:\git\msn_library\data\nina\20230806_gnps_grouped.parquet")

In [ ]:
df_grouped["compound_name", "entries", "any_phase"]